<a href="https://colab.research.google.com/github/adampotton/Group-3-gotta-catch-em-all-/blob/main/Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from PIL import Image
import os
from google.colab import drive
drive.mount('/content/drive')
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
root_directory = "/content/drive/My Drive/5Pokemon"

# Create a dictionary to store image arrays, using folder names as keys
image_arrays_dict = {}

# Define a common size for resizing images
common_size = (224, 224, 3)  # Common size for RGB images

# Loop through each folder in the root directory
for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)
    if os.path.isdir(folder_path):
        image_arrays = []
        for filename in os.listdir(folder_path):
            if filename.lower().endswith((".jpg", ".png")):
                image_path = os.path.join(folder_path, filename)
                img = Image.open(image_path).convert("RGB")  # Convert to RGB format
                img_resized = img.resize((common_size[1], common_size[0]), Image.ANTIALIAS)
                img_array = np.array(img_resized)
                image_arrays.append(img_array)
        image_arrays = np.array(image_arrays)
        image_arrays_dict[folder_name] = image_arrays

# Combine all image arrays into a single array
all_images = np.concatenate(list(image_arrays_dict.values()), axis=0)


for folder_name, image_array in image_arrays_dict.items():
    print(f"Shape of the image array for folder '{folder_name}': {image_array.shape}")

<ipython-input-30-896b65ba6408>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((common_size[1], common_size[0]), Image.ANTIALIAS)


Shape of the image array for folder 'Charmander': (51, 224, 224, 3)
Shape of the image array for folder 'Eevee': (37, 224, 224, 3)
Shape of the image array for folder 'Pikachu': (60, 224, 224, 3)
Shape of the image array for folder 'Squirtle': (52, 224, 224, 3)
Shape of the image array for folder 'Bulbasaur': (41, 224, 224, 3)


In [ ]:
pokemon_classes = ['Charmander', 'Eevee', 'Pikachu', 'Squirtle', 'Bulbasaur']
num_images_per_class = [51, 37, 60, 52, 41]
all_labels = np.concatenate([np.full(num, i) for i, num in enumerate(num_images_per_class)])


# Assuming you have the image arrays stored in 'all_images' and corresponding labels in 'all_labels'
# Make sure 'all_images' and 'all_labels' are NumPy arrays

# Convert data to PyTorch tensors
images_tensor = torch.tensor(all_images, dtype=torch.float32)
labels_tensor = torch.tensor(all_labels, dtype=torch.long)

# Create a PyTorch dataset
dataset = TensorDataset(images_tensor, labels_tensor)

# Create data loaders
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Number of classes for your Pokémon images
num_classes = 5

# Number of input channels in your Pokémon images
num_input_channels = 3  # Assuming RGB images

# Define a simpler model
model = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(32 * 56 * 56, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 4

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images.permute(0, 3, 1, 2))

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the accuracy after each epoch
    with torch.no_grad():
        total = 0
        correct = 0
        for images, labels in train_loader:
            outputs = model(images.permute(0, 3, 1, 2))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)

            # Adjust the size of true labels tensor dynamically
            correct += (predicted[:len(labels)] == labels).sum().item()

        accuracy = correct / total * 100
        print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%')


Epoch [1/4], Accuracy: 26.56%
Epoch [2/4], Accuracy: 34.85%
